In [2]:
import pandas as pd
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.database import Database

swissmetro = pd.read_csv("examples/swissmetro/swissmetro.dat", sep="\t")
database = Database("swissmetro", swissmetro)
globals().update(database.variables)

exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
database.remove(exclude)
database.data["CHOICE"] -= 1

ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 1)
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

TRAIN_TT = TRAIN_TT / 100.
SM_TT = SM_TT / 100.
CAR_TT = CAR_TT / 100.
TRAIN_CO = TRAIN_CO / 100.
SM_CO = SM_CO / 100.
CAR_CO = CAR_CO / 100.

# Definition of the utility functions
V1 = ASC_TRAIN + B_TIME * TRAIN_TT + B_COST * TRAIN_CO
V2 = ASC_SM + B_TIME * SM_TT + B_COST * SM_CO
V3 = ASC_CAR + B_TIME * CAR_TT + B_COST * CAR_CO

# Associate utility functions with the numbering of alternatives
V = {0: V1, 1: V2, 2: V3}

# Associate the availability conditions with the alternatives
av = {0: 1, 1: 1, 2: 1}

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = '01logit'

# Calculate the null log likelihood for reporting.
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(results)

              Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR   -0.631094  0.033841 -18.648792  0.000000e+00      0.035862   
ASC_TRAIN -1.258740  0.047261 -26.633967  0.000000e+00      0.050007   
B_COST     0.023146  0.002504   9.244533  0.000000e+00      0.001886   
B_TIME    -0.260554  0.035761  -7.286068  3.190781e-13      0.041584   

           Rob. t-test  Rob. p-value  
ASC_CAR     -17.597882  0.000000e+00  
ASC_TRAIN   -25.171216  0.000000e+00  
B_COST       12.270418  0.000000e+00  
B_TIME       -6.265668  3.712310e-10  

Results for model 01logit
Output file (HTML):			01logit.html
Nbr of parameters:		4
Sample size:			6768
Excluded data:			3960
Null log likelihood:		-7435.408
Init log likelihood:		-7435.408
Final log likelihood:		-6194.759
Likelihood ratio test (null):		2481.297
Rho square (null):			0.167
Rho bar square (null):			0.166
Likelihood ratio test (init):		2481.297
Rho square (init):			0.167
Rho bar square (init):			0.166
Akaike Information Criter